In [1]:
import os
import csv
import re

import urllib.request

import networkx
import pandas

import pandas as pd


import do_tools

In [4]:
base = 'https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/subsets/'
files = ['DO_cancer_slim.obo', 'DO_rare_slim.obo', 'TopNodes_DOcancerslim.obo']

for file in files:
    urllib.request.urlretrieve(base+file, 'download/'+file)

base = 'https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/'
file = 'HumanDO.obo'
urllib.request.urlretrieve(base+file, 'download/'+file);

('download/HumanDO.obo', <http.client.HTTPMessage at 0x7f8b9d09b048>)

In [2]:
path = os.path.join('download', 'HumanDO.obo')
do = do_tools.load_do(path)
dox = do_tools.do_to_networkx(do)

In [3]:
rdo = do_tools.load_do('download/DO_rare_slim.obo')

In [ ]:
slim = pd.read_table('data/slim-terms.tsv')
rdo_ids = [x for x in rdo.terms.keys()]

for doid in set(slim['doid']).intersection(set(rdo_ids)):
    rdo_ids.remove(doid)

In [4]:
rdox = do_tools.do_to_networkx(rdo)

In [24]:
for edge in rdox.edges():
    print(edge[0].id , ':', edge[0].name, '-----', edge[1].id , ':', edge[1].name)

DOID:0060670 : cerebral cavernous malformation 2 ----- DOID:0060669 : cerebral cavernous malformation
DOID:0060371 : autosomal dominant Parkinson disease 8 ----- DOID:14330 : Parkinson's disease
DOID:0060671 : cerebral cavernous malformation 3 ----- DOID:0060669 : cerebral cavernous malformation
DOID:0060290 : blepharophimosis-intellectual disability syndrome, SBBYS type ----- DOID:0060289 : Ohdo syndrome
DOID:0080182 : mixed fibrolamellar hepatocellular carcinoma ----- DOID:5015 : fibrolamellar carcinoma
DOID:3132 : porphyria cutanea tarda ----- DOID:3133 : acute porphyria
DOID:0060705 : X-linked lymphoproliferative syndrome 1 ----- DOID:0060704 : lymphoproliferative syndrome
DOID:0060438 : Cole-Carpenter syndrome ----- DOID:12347 : osteogenesis imperfecta
DOID:0080174 : bladder exstrophy ----- DOID:0080173 : bladder exstrophy-epispadias-cloacal exstrophy complex
DOID:0060706 : X-linked lymphoproliferative syndrome 2 ----- DOID:0060704 : lymphoproliferative syndrome
DOID:0060368 : aut

just going to keep the more general of the two terms...

In [38]:
for edge in rdox.edges():
    rdo_ids.remove(edge[0].id)

In [39]:
len(rdo_ids)

467

In [40]:
rdo_names = [rdo[doid].name for doid in rdo_ids]

In [42]:
slim.head(2)

,doid,name,source,pathophysiology
0,DOID:2531,hematologic cancer,DOcancerslim,neoplastic
1,DOID:1319,brain cancer,DOcancerslim,neoplastic


In [43]:
r_slim = pd.DataFrame()
r_slim['doid'] = rdo_ids
r_slim['name'] = rdo_names
r_slim['source'] = 'DOrareslim'
r_slim.head()

,doid,name,source
0,DOID:0060301,type I complement component 8 deficiency,DOrareslim
1,DOID:9563,bronchiectasis,DOrareslim
2,DOID:0060295,complement component 2 deficiency,DOrareslim
3,DOID:3652,Leigh disease,DOrareslim
4,DOID:10825,essential hypertension,DOrareslim


In [44]:
slim_out = pd.concat([slim, r_slim])

In [46]:
slim_out = slim_out.fillna('undetermined')

In [47]:
slim_out.to_csv('data/slim-terms1.tsv', index=False, sep='\t')